# Day 17: Conway Cubes

[*Advent of Code 2020 day 17*](https://adventofcode.com/2020/day/17) and [*solution megathread*](https://redd.it/keqsfa)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2020/17/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2020%2F17%2Fcode.ipynb)

In [4]:
from IPython.display import HTML
import sys
sys.path.append('../../')
import common

downloaded = common.refresh()
%store downloaded >downloaded

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [5]:
HTML(downloaded['part1'])

## Boilerplate

Let's try using [pycodestyle_magic](https://github.com/mattijn/pycodestyle_magic) with pycodestyle (flake8 stopped working for me in VS Code Jupyter). Now how does type checking work?

In [6]:
%load_ext pycodestyle_magic

In [7]:
%pycodestyle_on

In [18]:
testdata = """.#.
..#
###""".splitlines()

inputdata = """##..####
.###....
#.###.##
#....#..
...#..#.
#.#...##
..#.#.#.
.##...#.""".splitlines()

In [22]:
from itertools import product


class ConwayCubes(object):
    def __init__(self, data):
        self.active = set()
        self.neighbors = dict()
        self.min_z = self.max_z = z = 0
        self.min_y = self.min_x = 0
        if len(data) == 0:
            self.max_y = self.max_x = 0
            return
        self.max_y = self.max_x = len(data) - 1
        for y, line in enumerate(data):
            for x, cube in enumerate(line):
                if cube == '#':
                    self.activate((z, y, x))

    def __repr__(self):
        layers = []
        for z in range(self.min_z, self.max_z + 1):
            layer = []
            layer.append(f'z={z}')
            for y in range(self.min_y, self.max_y + 1):
                layer.append(
                    ''.join(['#' if (z, y, x) in self.active else '.'
                             for x in range(self.min_x, self.max_x + 1)])
                )
            layers.append('\n'.join(layer))
        return '\n\n'.join(layers)

    def add_coords(a, b):
        return tuple([sum(ab) for ab in zip(a, b)])

    def activate(self, coord):
        for offset in product(range(-1, 2), range(-1, 2), range(-1, 2)):
            if offset == (0, 0, 0):
                continue
            n_coord = ConwayCubes.add_coords(coord, offset)
            if n_coord not in self.neighbors:
                # print(f'Defining new neighbor {n_coord}')
                self.neighbors[n_coord] = 1
            else:
                # print(f'Incrementing neighbor {n_coord}')
                self.neighbors[n_coord] += 1

            if coord[0] < self.min_z:
                self.min_z = coord[0]
            elif coord[0] > self.max_z:
                self.max_z = coord[0]
            if coord[1] < self.min_y:
                self.min_y = coord[1]
            elif coord[1] > self.max_y:
                self.max_y = coord[1]
            if coord[2] < self.min_x:
                self.min_x = coord[2]
            elif coord[2] > self.max_x:
                self.max_x = coord[2]

            self.active.add(coord)

    # If a cube is active and exactly 2 or 3 of its neighbors are also active,
    # the cube remains active. Otherwise, the cube becomes inactive
    # If a cube is inactive but exactly 3 of its neighbors are active,
    # the cube becomes active. Otherwise, the cube remains inactive
    def iterate(self):
        cc_next = ConwayCubes('')
        for a_coord in self.active:
            if (a_coord in self.neighbors and
                    2 <= self.neighbors[a_coord] <= 3):
                cc_next.activate(a_coord)
        for n_coord in self.neighbors.keys():
            if (n_coord not in self.active and
                    self.neighbors[n_coord] == 3):
                cc_next.activate(n_coord)
        return cc_next

In [10]:
cc = ConwayCubes(testdata)
# print(cc)
for i in range(1, 7):
    cc = cc.iterate()
    # print(f'\nAfter {i} cycles:\n')
    # print(cc)
print(len(cc.active))

112


In [11]:
# %%timeit -n1 -r1
cc = ConwayCubes(inputdata)
# print(cc)
for i in range(1, 7):
    cc = cc.iterate()
    # print(f'\nAfter {i} cycles:\n')
    # print(cc)
print(len(cc.active))

209


In [12]:
HTML(downloaded['part1_footer'])

## Part Two

In [13]:
HTML(downloaded['part2'])

In [26]:
class ConwayHyperCubes(object):
    def __init__(self, data):
        self.active = set()
        self.neighbors = dict()
        self.minimums = [0, 0, 0, 0]
        self.maximums = [0, 0, 0, 0]
        if len(data) == 0:
            return
        self.maximums[2] = self.maximums[3] = len(data) - 1
        w = z = 0
        for y, line in enumerate(data):
            for x, cube in enumerate(line):
                if cube == '#':
                    self.activate((w, z, y, x))

    def __repr__(self):
        layers = []
        for w, z in product(range(self.minimums[0], self.maximums[0] + 1),
                            range(self.minimums[1], self.maximums[1] + 1)):
            layer = []
            layer.append(f'z={z}, w={w}')
            for y in range(self.minimums[2], self.maximums[2] + 1):
                layer.append(
                    ''.join(['#' if (w, z, y, x) in self.active else '.'
                             for x in range(self.minimums[3],
                                            self.maximums[3] + 1)])
                    )
            layers.append('\n'.join(layer))
        return '\n\n'.join(layers)

    def add_coords(a, b):
        return tuple([sum(ab) for ab in zip(a, b)])

    def activate(self, coord):
        for offset in product(range(-1, 2),
                              range(-1, 2),
                              range(-1, 2),
                              range(-1, 2)):
            if offset == (0, 0, 0, 0):
                continue
            n_coord = ConwayCubes.add_coords(coord, offset)
            if n_coord not in self.neighbors:
                # print(f'Defining new neighbor {n_coord}')
                self.neighbors[n_coord] = 1
            else:
                # print(f'Incrementing neighbor {n_coord}')
                self.neighbors[n_coord] += 1

            self.minimums = list(map(min, zip(coord, self.minimums)))
            self.maximums = list(map(max, zip(coord, self.maximums)))
            self.active.add(coord)

    # If a cube is active and exactly 2 or 3 of its neighbors are also active,
    # the cube remains active. Otherwise, the cube becomes inactive
    # If a cube is inactive but exactly 3 of its neighbors are active,
    # the cube becomes active. Otherwise, the cube remains inactive
    def iterate(self):
        cc_next = ConwayHyperCubes('')
        for a_coord in self.active:
            if (a_coord in self.neighbors and
                    2 <= self.neighbors[a_coord] <= 3):
                cc_next.activate(a_coord)
        for n_coord in self.neighbors.keys():
            if (n_coord not in self.active and
                    self.neighbors[n_coord] == 3):
                cc_next.activate(n_coord)
        return cc_next

In [27]:
cc = ConwayHyperCubes(testdata)
# print(cc)
for i in range(1, 7):
    cc = cc.iterate()
    # print(f'\nAfter {i} cycles:\n')
    # print(cc)
print(len(cc.active))

848


In [28]:
# %%timeit -n1 -r1
cc = ConwayHyperCubes(inputdata)
# print(cc)
for i in range(1, 7):
    cc = cc.iterate()
    # print(f'\nAfter {i} cycles:\n')
    # print(cc)
print(len(cc.active))

1492


In [29]:
HTML(downloaded['part2_footer'])